# AutoML: Machine Learning Automatico

El proceso de Machine Learning se puede automatizar un poco. Hay varios servicios en internet que lo hacen y tambien varias librerias. Aqui veremos las siguientes librerias: 
- [FLAML](https://microsoft.github.io/FLAML/docs/Getting-Started/)
- [AutoGluon](https://auto.gluon.ai/)
- [PyCaret](https://pycaret.org/)



## FLAML

Comencemos con FLAML para automatizar solo el proceso de seleccion del algoritmo de machine learning y los hiperparametros

In [ ]:
!pip install "flaml[automl]" -U -q

Como vimos en la unidad anterior el proceso de Machine Learning culmina entrenando distintas cajas negras con distintos algoritmos y ajustando los parametros de dichos algoritmos para que el resultado del entrenamiento sea el mejor posible. Ahora veremos como utilizar la libreria FLAML para automatizar este proceso.

Primero utilicemos los datos que hemos preprocesado de nuestra base de datos de pacientes con diabetes:

In [ ]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df_train = pd.read_csv('https://raw.githubusercontent.com/amiune/freecodingtour/main/cursos/espanol/datascience/data/diabetes/diabetes_train_procesado.csv')
df_test = pd.read_csv('https://raw.githubusercontent.com/amiune/freecodingtour/main/cursos/espanol/datascience/data/diabetes/diabetes_test_procesado.csv')

In [ ]:
X_train = df_train.loc[:, df_train.columns != "diabetes"]
y_train = df_train.loc[:, "diabetes"]

X_train.head()

Ahora utilicemos la libreria FLAML a la cual le daremos la tabla de entrenamiento separada en X e y al igual que con cualquier algoritmo de sklearn pero ademas especificaremos la tarea a realizar con el parametro task="classification" y el tiempo de computo que le daremos a la libreria para que elcuentre la mejor caja negra posible, en este caso utilizaremos time_budget=10 para darle 10 segundos.

In [ ]:
from flaml import AutoML
automl = AutoML()
automl.fit(X_train, y_train, task="classification", time_budget=10)

Una vez que la libreria ha terminado de entrenar distintos modelos podemos ver cual ha sido el mejor (best_estimator), cuales son sus parametros (best_config) y el resultado de su funcion de error (best_loss)

In [ ]:
automl.best_estimator

In [ ]:
automl.best_config

In [ ]:
automl.best_loss

Finalmente podemos utilizar nuestra tabla de prueba para ver que tan bien se desempeña en ella este modelo elegido por la libreria 

In [ ]:
X_test = df_test.loc[:, df_test.columns != "diabetes"]
y_test = df_test.loc[:, "diabetes"]

In [ ]:
from sklearn.metrics import accuracy_score

def calcular_accuracy_train_val(clf, X_train, y_train, X_val, y_val):
    y_train_pred = clf.predict(X_train)
    print("Entrenamiento accuracy:",accuracy_score(y_train, y_train_pred))
    y_val_pred = clf.predict(X_val)
    print("Validacion accuracy:",accuracy_score(y_val, y_val_pred))
    return clf

In [ ]:
clf = calcular_accuracy_train_val(automl, X_train, y_train, X_test, y_test)

## AutoGluon

Ahora utilizaremos AutoGluon para automatizar solo el proceso de seleccion del algoritmo de machine learning y los hiperparametros

In [ ]:
!pip install -U autogluon.tabular


In [ ]:
from autogluon.tabular import TabularDataset, TabularPredictor

Cargamos los datos de entrenamiento en la clase especial de AutoGluon llamada TabularDataset

In [ ]:
df_ana_train = pd.read_csv('https://raw.githubusercontent.com/amiune/freecodingtour/main/cursos/espanol/datascience/data/diabetes/diabetes_train_analisis.csv')
df_inf_train = pd.read_csv('https://raw.githubusercontent.com/amiune/freecodingtour/main/cursos/espanol/datascience/data/diabetes/diabetes_train_info.csv')
df_train = pd.merge(df_ana_train, df_inf_train, on="id")
df_train = df_train.drop("id", axis=1)
train_data = TabularDataset(df_train)

Entrenamos varios modelos

In [ ]:
predictor = TabularPredictor(
    label='diabetes',
    eval_metric='accuracy'
).fit(
    train_data,
    time_limit=120
)

Cargamos los datos de prueba

In [ ]:
df_ana_test = pd.read_csv('https://raw.githubusercontent.com/amiune/freecodingtour/main/cursos/espanol/datascience/data/diabetes/diabetes_test_analisis.csv')
df_inf_test = pd.read_csv('https://raw.githubusercontent.com/amiune/freecodingtour/main/cursos/espanol/datascience/data/diabetes/diabetes_test_info.csv')
df_test = pd.merge(df_ana_test, df_inf_test, on="id")
df_test = df_test.drop("id", axis=1)
test_data = TabularDataset(df_test)

Evaluamos los modelos y comparamos sus metricas

In [ ]:
predictor.evaluate(test_data)

In [ ]:
predictor.leaderboard(test_data)

In [ ]:
fi = predictor.feature_importance(train_data)
print(fi)

Finalmente hacemos las predicciones que queremos

In [ ]:
y_pred = predictor.predict(test_data.drop(columns=['diabetes']))
y_pred.head()

## Como especificar parametros

In [ ]:
predictor = TabularPredictor(label=label).fit(
    train_data,
    time_limit=600,           # 10 minutes
    presets="medium_quality", # or "best_quality", "high_quality_fast_inference_only_refit"
)

In [ ]:
predictor = TabularPredictor(
    label=label,
    problem_type="multiclass",    # or "binary", "regression"
    eval_metric="accuracy",       # or "roc_auc", "log_loss", "rmse", ...
).fit(train_data)

## PyCaret

Similar a AutoGluon pero un poco mas simple

In [ ]:
from pycaret.classification import *
s = setup(df_train, target='diabetes', session_id=42)

In [ ]:
best = compare_models()

In [ ]:
predictions = predict_model(best, data = df_test)

# Fin: [Volver al contenido del curso](https://www.freecodingtour.com/cursos/espanol/datascience/datascience.html)